In [1]:
import pandas as pd
import os
import glob
import re
from nats_client import open_msg_sender,publish_message_async
import json

In [2]:
csv_files = glob.glob('./datas/tweets/*.csv')

In [3]:
dfs = [pd.read_csv(fl) for fl in csv_files]

In [4]:
all_dfs = pd.concat(dfs)
all_dfs = all_dfs.drop_duplicates('id')

In [5]:
all_dfs2 = all_dfs[['id','text','author','media','url','created_on']]

In [6]:
all_dfs2=  all_dfs2.dropna()

In [7]:
# 过期时间
all_dfs2['created_on'] = pd.to_datetime(all_dfs2['created_on'])
threshold_date = pd.Timestamp.now(tz='UTC') - pd.Timedelta(days=1)
all_dfs2 = all_dfs2[all_dfs2['created_on']>threshold_date]

In [8]:
all_dfs3 = all_dfs2.dropna()


In [9]:
len(all_dfs3)

8146

In [10]:
ethereum_address_pattern = r'0x([A-Fa-f0-9]{40})'
tags = ['whitelist', 'airdrop','whitelists','wl']
# 匹配标签
tags_pattern = r'\b(' + '|'.join(tags) + r')\b'

In [11]:
all_dfs3['ca'] = all_dfs3['text'].str.extract(ethereum_address_pattern, flags=re.MULTILINE)

In [12]:
all_dfs3['other'] = all_dfs3['text'].str.extract(tags_pattern,flags=re.IGNORECASE)

In [13]:
all_dfs3 = all_dfs3[~all_dfs3['ca'].isna() | ~all_dfs3['other'].isna()]

In [14]:
all_dfs3= all_dfs3.sort_values(['created_on','ca'],ascending=False)

In [18]:
all_dfs3.reset_index()

,index,id,text,author,media,url,created_on,ca,other
0,0,1.779955e+18,I've already done a complete tutorial on @synt...,"User(id=1279135957959262208, username=0xFastLi...",[],https://twitter.com/0xFastLife/status/17799546...,2024-04-15 19:27:16+00:00,NaN,airdrop
1,0,1.779944e+18,OKX New Chain - OKB XLayer\n\nBuy OKB on ETH M...,"User(id=1384916451312771072, username=0xBlubbe...",[],https://twitter.com/0xBlubberfurry/status/1779...,2024-04-15 18:43:41+00:00,75231F58b43240C9718Dd58B4967c5114342a86c,NaN
2,0,1.779934e+18,GameGPTofficial's $DUEL is a game-changer! 🥷🏿✝...,"User(id=1629333565, username=0xBispo, name=Def...","[Media(id=1779933868180566016, type=photo)]",https://twitter.com/0xBispo/status/17799338750...,2024-04-15 18:04:40+00:00,NaN,Airdrop
3,2,1.779926e+18,Jumper Exchange is great and I can't wait to t...,"User(id=819748581984833537, username=0xPajke, ...",[],https://twitter.com/0xPajke/status/17799257837...,2024-04-15 17:32:30+00:00,NaN,airdrop
4,1,1.779910e+18,MENDI finance is the one & only protocol on @L...,"User(id=1179019608738713600, username=0x366e, ...","[Media(id=1779890767609081856, type=photo)]",https://twitter.com/0x366e/status/177991029981...,2024-04-15 16:30:59+00:00,NaN,airdrop
5,3,1.779909e+18,Bought $BLUE @bluebasecoin on Base! 🟦 🥷🏿\n\n$B...,"User(id=1629333565, username=0xBispo, name=Def...","[Media(id=1779909350305189889, type=photo)]",https://twitter.com/0xBispo/status/17799093579...,2024-04-15 16:27:14+00:00,Fd9fa4F785331cE88b5Af8994A047bA087c705D8,NaN
6,2,1.779885e+18,Our governance token $ETHFI is going multi-cha...,"User(id=1254766283935096832, username=0xKATYA,...",[],https://twitter.com/0xKATYA/status/17798853087...,2024-04-15 14:51:40+00:00,NaN,airdrop
7,2,1.779853e+18,. @0xsebastiena just annouced leaving zkSync a...,"User(id=1279135957959262208, username=0xFastLi...",[],https://twitter.com/0xFastLife/status/17798530...,2024-04-15 12:43:38+00:00,NaN,airdrop
8,4,1.779786e+18,"𝗔𝗜𝗥𝗗𝗥𝗢𝗣\n\n@elixir launched Apothecary, a camp...","User(id=1279135957959262208, username=0xFastLi...","[Media(id=1779556959563522048, type=photo)]",https://twitter.com/0xFastLife/status/17797855...,2024-04-15 08:15:17+00:00,NaN,airdrop
9,0,1.779696e+18,"Merlin Penguins ""梅林巫师企鹅“ 是梅林生态的第一个M404 协议，衔接bt...","User(id=1315492227175477250, username=0xJamesX...","[Media(id=1779694537642065920, type=photo)]",https://twitter.com/0xJamesXXX/status/17796960...,2024-04-15 02:19:28+00:00,NaN,WL


In [19]:
all_dfs3[['url','created_on','ca','other','text']].to_csv('ca.csv')

In [44]:
for index,row in all_dfs3.fillna('').head(10).iterrows():
    await publish_message_async('tgmsg',json.dumps({
        "type": "new_tweet",
        "from_account": '111',
        "tweet": {
            "text": row[1],
            "address": row[6],
            "tg": "",
            "url": row[4],
            "tags": row[7]
        }
    }))


/var/folders/dm/h6ld_rrn2js8_khbw9pstjh80000gn/T/ipykernel_75570/2159292138.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "text": row[1],
/var/folders/dm/h6ld_rrn2js8_khbw9pstjh80000gn/T/ipykernel_75570/2159292138.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "address": row[6],
/var/folders/dm/h6ld_rrn2js8_khbw9pstjh80000gn/T/ipykernel_75570/2159292138.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "url": row[4],
/v

In [24]:
txt = all_dfs3[['text']].head(1)

In [36]:
all_dfs3[all_dfs3['ca'].isna() == False][['text']].to_csv('txt')

In [46]:
len(dfs)

3662

In [24]:
all_dfs3['xl'] = all_dfs3['text'].str.extract(r'\b(x layer)\b',flags=re.IGNORECASE)

In [26]:
all_dfs3.sort_values(['xl'])

,id,text,author,media,url,created_on,ca,other,xl
0,1779962552924905639,You need $OKB (ERC20) to bridge to X Layer.\n\...,"User(id=1485550409305530371, username=neemdot,...",[],https://twitter.com/neemdot/status/17799625529...,2024-04-15 19:58:37+00:00,734ddad5e0cdab730dA30c580f32AEA83BEAb218,NaN,X Layer
0,1779943696982577408.0,OKX New Chain - OKB XLayer\n\nBuy OKB on ETH M...,"User(id=1384916451312771072, username=0xBlubbe...",[],https://twitter.com/0xBlubberfurry/status/1779...,2024-04-15 18:43:41+00:00,75231F58b43240C9718Dd58B4967c5114342a86c,NaN,X Layer
0,1780042498414403863,Aggressively farming $PARAM again.\n\nIf this ...,"User(id=1304166174238834693, username=Joshyysm...","[Media(id=1780042491816779776, type=photo)]",https://twitter.com/Joshyysmartt/status/178004...,2024-04-16 01:16:17+00:00,NaN,airdrop,NaN
0,1780036459463884821,ARKHAM 2.0 Airdrop 🪂😙\n\n⏰ Time: 1sec\n💰 Cost...,"User(id=1518056494520053761, username=binadam6...","[Media(id=1780036452115533824, type=photo)]",https://twitter.com/binadam61/status/178003645...,2024-04-16 00:52:18+00:00,NaN,Airdrop,NaN
0,1780032779641606400.0,Is @pacmoon_ the least over-farmed airdrop?\n\...,"User(id=1593822657069432834, username=Zaddy_NF...","[Media(id=1780031678573875200, type=photo)]",https://twitter.com/Zaddy_NFT/status/178003277...,2024-04-16 00:37:40+00:00,NaN,airdrop,NaN
...,...,...,...,...,...,...,...,...,...
12,1779809352246853854,Have you been active in the Arbitrum ecosystem...,"User(id=1518056494520053761, username=binadam6...","[Media(id=1779809349126225921, type=photo)]",https://twitter.com/binadam61/status/177980935...,2024-04-15 09:49:51+00:00,NaN,Airdrop,NaN
7,1779806594542215278,Günaydın yaraları sarmaya başladık güzel haber...,"User(id=1748009257205903360, username=theFanto...","[Media(id=1779792910386364416, type=photo), Me...",https://twitter.com/theFantomgoat/status/17798...,2024-04-15 09:38:54+00:00,NaN,airdrop,NaN
8,1779805760735605225,$SPEEDY is obviously going 100m+ on $FTM and w...,"User(id=1748009257205903360, username=theFanto...",[],https://twitter.com/theFantomgoat/status/17798...,2024-04-15 09:35:35+00:00,NaN,airdrop,NaN
11,1779805701008916480.0,🦥 @slerfsol will be launching their new NFT on...,"User(id=1454671970956500993, username=LeoStudi...","[Media(id=1779100972326801408, type=photo)]",https://twitter.com/LeoStudio_LS/status/177980...,2024-04-15 09:35:20+00:00,NaN,WL,NaN
